In [181]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
import re
from collections import Counter
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import add
from keras.layers import concatenate
from keras.layers import Concatenate
from keras.layers import Input
from keras.layers import Conv1D,Convolution1D
from keras.layers import MaxPool1D,MaxPooling1D
from keras.layers import SpatialDropout1D
from keras.optimizers import Adam
from keras.objectives import binary_crossentropy
from sklearn.model_selection import train_test_split
from keras.layers import Merge

In [108]:
word_dict = {}
word_counter = Counter()
model_path = "../models/"

In [109]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ", string)
    string = re.sub(r"\)", " ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r",", " ", string)
    string = re.sub(r"'", " ", string)
    string = re.sub(r"\[", " ", string)
    string = re.sub(r"\]", " ", string)
    return string.strip().lower()

In [110]:
def create_count(dict_file):
    print (type(dict_file))
    count = 0
    for line in dict_file:
        for word in (clean_str(line)).split():
            if word not in word_dict.keys():
                word_counter[word] += 1

In [111]:
with open("./../fishing_data/pos.txt", "r", encoding="utf8") as pos_file:
    create_count(pos_file)
with open("./../fishing_data/neg.txt", "r", encoding="utf8") as pos_file:
    create_count(pos_file)
with open("./../fishing_data/test/pos.txt", "r", encoding="utf8") as pos_file:
    create_count(pos_file)
with open("./../fishing_data/test/neg.txt", "r", encoding="utf8") as pos_file:
     create_count(pos_file)

<class '_io.TextIOWrapper'>
<class '_io.TextIOWrapper'>
<class '_io.TextIOWrapper'>
<class '_io.TextIOWrapper'>


In [112]:
count = 0
for word in word_counter.most_common():
    count += 1
    word_dict[word[0]] = count

In [113]:
idx2word = {v: k for k, v in word_dict.items()}

In [114]:
def line_mapper(input_file):
    output_string = []
    for line in input_file:
        output_line = []
        for word in (clean_str(line)).split():
            output_line.append(word_dict[word])
        output_string.append(output_line)
    return output_string

In [115]:
train_pos = [];
train_neg = [];
test_pos = [];
test_neg = [];

labels_test = [];
with open("./../fishing_data/pos.txt", "r", encoding="utf8") as pos_file:
    train_pos = line_mapper(pos_file)
with open("./../fishing_data/neg.txt", "r", encoding="utf8") as pos_file:
    train_neg = line_mapper(pos_file)
with open("./../fishing_data/test/pos.txt", "r", encoding="utf8") as pos_file:
    test_pos = line_mapper(pos_file)
with open("./../fishing_data/test/neg.txt", "r", encoding="utf8") as pos_file:
    test_neg = line_mapper(pos_file)

In [116]:
labels_train_pos = [1] * len(train_pos)
labels_train_neg = [0] * len(train_neg)
labels_test_pos = [1] * len(test_pos)
labels_test_neg = [0] * len(test_neg)

In [117]:
print("the number of train positive",len(train_pos))
print("the number of train negative",len(train_neg))
print("the number of test positive",len(test_pos))
print("the number of test negative",len(test_neg))

the number of train positive 4338
the number of train negative 4392
the number of test positive 722
the number of test negative 732


In [118]:
train_set = train_pos + train_neg
test_set = test_pos + test_neg
train_labels =  labels_train_pos + labels_train_neg
test_labels = labels_test_pos + labels_test_neg
print("the total number of training examples",len(train_set))
print("the total number of trainng labels",len(train_labels))
print("the total number of testing examples",len(test_set))
print("the total number of testing labels",len(test_labels))

the total number of training examples 8730
the total number of trainng labels 8730
the total number of testing examples 1454
the total number of testing labels 1454


In [119]:
vocab_size = 5000
total_data = train_set + test_set;
total_labels = train_labels + test_labels;

#trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in train_set]
#test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in test_set]
t_data = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in total_data]

In [120]:
print(len(t_data))

10184


In [121]:
X_train_val, X_test, y_train_val, y_test = train_test_split(t_data, total_labels, test_size=0.3, random_state=42)

In [122]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=42)

In [123]:
min_len = min(map(len, t_data))
max_len = max(map(len,t_data))
avg_len = sum(map(len,t_data)) / len(t_data)
print("the minimum length is:",min_len)
print("the maximum length is:",max_len)
print("the average length is:",avg_len)

the minimum length is: 2
the maximum length is: 1012
the average length is: 96.83356245090337


In [124]:
seq_len = 100

trn = sequence.pad_sequences(X_train, maxlen=seq_len, value=0)
val = sequence.pad_sequences(X_val, maxlen=seq_len, value=0)
test = sequence.pad_sequences(X_test, maxlen=seq_len, value=0)

In [125]:
trn.shape

(4989, 100)

# Simple Dense model

In [126]:
model = Sequential([
    Embedding(vocab_size, 200, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')])

In [127]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 200)          1000000   
_________________________________________________________________
flatten_3 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               2000100   
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 3,000,201.0
Trainable params: 3,000,201.0
Non-trainable params: 0.0
_________________________________________________________________


In [128]:
model.fit(trn, y_train, validation_data=(val, y_val), epochs=5, batch_size=64)

Train on 4989 samples, validate on 2139 samples
Epoch 1/5
4989/4989 [==============================] - 1s - loss: 0.4081 - acc: 0.8104 - val_loss: 0.2418 - val_acc: 0.9126
Epoch 2/5
4989/4989 [==============================] - 1s - loss: 0.1193 - acc: 0.9585 - val_loss: 0.2494 - val_acc: 0.9098
Epoch 3/5
4989/4989 [==============================] - 1s - loss: 0.0318 - acc: 0.9928 - val_loss: 0.2768 - val_acc: 0.9056
Epoch 4/5
4989/4989 [==============================] - 1s - loss: 0.0093 - acc: 0.9982 - val_loss: 0.3096 - val_acc: 0.9121
Epoch 5/5
4989/4989 [==============================] - 1s - loss: 0.0033 - acc: 0.9994 - val_loss: 0.3499 - val_acc: 0.9051


In [129]:
loss_and_metrics = model.evaluate(test, y_test, batch_size=64);
print("the accuracy on test set is:",loss_and_metrics[1])

2560/3056 [========================>.....] - ETA: 0sthe accuracy on test set is: 0.889397906071


In [65]:
model.save_weights(model_path + 'dense.h5')

# Convolution neural network

In [130]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    SpatialDropout1D(0.2),
    Convolution1D(64, 5, padding='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [131]:

conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [132]:
conv1.fit(trn, y_train, validation_data=(test, y_test), epochs=6, batch_size=64)

Train on 4989 samples, validate on 3056 samples
Epoch 1/6
4989/4989 [==============================] - 1s - loss: 0.5196 - acc: 0.7236 - val_loss: 0.2775 - val_acc: 0.8973
Epoch 2/6
4989/4989 [==============================] - 0s - loss: 0.2226 - acc: 0.9204 - val_loss: 0.2566 - val_acc: 0.9025
Epoch 3/6
4989/4989 [==============================] - 0s - loss: 0.1614 - acc: 0.9465 - val_loss: 0.2585 - val_acc: 0.9051
Epoch 4/6
4989/4989 [==============================] - 0s - loss: 0.1070 - acc: 0.9645 - val_loss: 0.3092 - val_acc: 0.8986
Epoch 5/6
4989/4989 [==============================] - 0s - loss: 0.0862 - acc: 0.9743 - val_loss: 0.3252 - val_acc: 0.9022
Epoch 6/6
4989/4989 [==============================] - 0s - loss: 0.0555 - acc: 0.9856 - val_loss: 0.3790 - val_acc: 0.8969


In [133]:
loss_and_metrics = conv1.evaluate(test, y_test, batch_size=64);
print("the accuracy on test set is:",loss_and_metrics[1])

2368/3056 [======================>.......] - ETA: 0sthe accuracy on test set is: 0.89692408377


In [134]:
model.save_weights(model_path + 'simple_conv.h5')

# Pretrained weights with glove embedding

In [145]:
EMBEDDING_DIM = 50
seq_len = 100
vocab_size = 5000

In [136]:
def load_glove_index(loc):
    f = open(loc,encoding="utf8")
    embeddings_index = {}
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

In [137]:
embeddings_index = load_glove_index("../glove/" + 'glove.6B.50d.txt')


In [138]:
embeddings_index['the']

array([  4.18000013e-01,   2.49679998e-01,  -4.12420005e-01,
         1.21699996e-01,   3.45270008e-01,  -4.44569997e-02,
        -4.96879995e-01,  -1.78619996e-01,  -6.60229998e-04,
        -6.56599998e-01,   2.78430015e-01,  -1.47670001e-01,
        -5.56770027e-01,   1.46579996e-01,  -9.50950012e-03,
         1.16579998e-02,   1.02040000e-01,  -1.27920002e-01,
        -8.44299972e-01,  -1.21809997e-01,  -1.68009996e-02,
        -3.32789987e-01,  -1.55200005e-01,  -2.31309995e-01,
        -1.91809997e-01,  -1.88230002e+00,  -7.67459989e-01,
         9.90509987e-02,  -4.21249986e-01,  -1.95260003e-01,
         4.00710011e+00,  -1.85939997e-01,  -5.22870004e-01,
        -3.16810012e-01,   5.92130003e-04,   7.44489999e-03,
         1.77780002e-01,  -1.58969998e-01,   1.20409997e-02,
        -5.42230010e-02,  -2.98709989e-01,  -1.57490000e-01,
        -3.47579986e-01,  -4.56370004e-02,  -4.42510009e-01,
         1.87849998e-01,   2.78489990e-03,  -1.84110001e-01,
        -1.15139998e-01,

In [139]:
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in word_dict.items():
    if i >= vocab_size:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [146]:
embedding_layer = Embedding(vocab_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=seq_len,
                            trainable=False)


In [168]:
sequence_input = Input(shape=(seq_len,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Convolution1D(64, 5, activation='relu',padding='same')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Flatten()(x)
x = Dense(100, activation='relu')(x)
preds = Dense(1, activation='sigmoid')(x)



In [169]:
conv2 = Model(sequence_input, preds)
conv2.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
conv2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 100)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 100, 50)           250000    
_________________________________________________________________
conv1d_48 (Conv1D)           (None, 100, 64)           16064     
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 20, 64)            0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 1280)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 100)               128100    
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 101       
Total para

In [170]:
conv2.fit(trn, y_train, validation_data=(test, y_test), epochs=6, batch_size=64)

Train on 4989 samples, validate on 3056 samples
Epoch 1/6
4989/4989 [==============================] - 2s - loss: 0.3837 - acc: 0.8286 - val_loss: 0.3116 - val_acc: 0.8940
Epoch 2/6
4989/4989 [==============================] - 0s - loss: 0.2505 - acc: 0.9030 - val_loss: 0.2769 - val_acc: 0.8884
Epoch 3/6
4989/4989 [==============================] - 0s - loss: 0.1950 - acc: 0.9246 - val_loss: 0.2666 - val_acc: 0.8917
Epoch 4/6
4989/4989 [==============================] - 0s - loss: 0.1479 - acc: 0.9445 - val_loss: 0.3024 - val_acc: 0.8969
Epoch 5/6
4989/4989 [==============================] - 0s - loss: 0.1000 - acc: 0.9651 - val_loss: 0.3449 - val_acc: 0.8819
Epoch 6/6
4989/4989 [==============================] - 0s - loss: 0.0520 - acc: 0.9848 - val_loss: 0.3831 - val_acc: 0.8878


In [171]:
model.layers[1].trainable=True

In [172]:
conv2.fit(trn, y_train, validation_data=(test, y_test), epochs=6, batch_size=64)

Train on 4989 samples, validate on 3056 samples
Epoch 1/6
4989/4989 [==============================] - 0s - loss: 0.0310 - acc: 0.9942 - val_loss: 0.3885 - val_acc: 0.8946
Epoch 2/6
4989/4989 [==============================] - 0s - loss: 0.0141 - acc: 0.9974 - val_loss: 0.4452 - val_acc: 0.8923
Epoch 3/6
4989/4989 [==============================] - 0s - loss: 0.0078 - acc: 0.9988 - val_loss: 0.4873 - val_acc: 0.8973
Epoch 4/6
4989/4989 [==============================] - 0s - loss: 0.0045 - acc: 0.9994 - val_loss: 0.5410 - val_acc: 0.8933
Epoch 5/6
4989/4989 [==============================] - 0s - loss: 0.0026 - acc: 0.9998 - val_loss: 0.5553 - val_acc: 0.8930
Epoch 6/6
4989/4989 [==============================] - 0s - loss: 0.0018 - acc: 0.9998 - val_loss: 0.5801 - val_acc: 0.8953


## Multi-size CNN

In [183]:
graph_in = Input ((vocab_size, 50))
convs = [ ] 
for fsz in range (3, 6): 
    x = Convolution1D(64, fsz, padding='same', activation="relu")(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
out = concatenate(convs) 
graph = Model(graph_in, out) 

In [184]:
conv3 = Sequential ([
    Embedding(vocab_size, 50, input_length=seq_len, weights=[embedding_matrix]),
    SpatialDropout1D(0.2),
    graph,
    Dropout (0.5),
    Dense (100, activation="relu"),
    Dropout (0.7),
    Dense (1, activation='sigmoid')
    ])

In [186]:
conv3.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [188]:
conv3.fit(trn, y_train, validation_data=(test, y_test), epochs=10, batch_size=64)

Train on 4989 samples, validate on 3056 samples
Epoch 1/10
4989/4989 [==============================] - 1s - loss: 0.1503 - acc: 0.9435 - val_loss: 0.2561 - val_acc: 0.9103
Epoch 2/10
4989/4989 [==============================] - 1s - loss: 0.1248 - acc: 0.9517 - val_loss: 0.2894 - val_acc: 0.9139
Epoch 3/10
4989/4989 [==============================] - 1s - loss: 0.1227 - acc: 0.9513 - val_loss: 0.2699 - val_acc: 0.9149
Epoch 4/10
4989/4989 [==============================] - 1s - loss: 0.1005 - acc: 0.9617 - val_loss: 0.2769 - val_acc: 0.9100
Epoch 5/10
4989/4989 [==============================] - 1s - loss: 0.0897 - acc: 0.9701 - val_loss: 0.2922 - val_acc: 0.9175
Epoch 6/10
4989/4989 [==============================] - 1s - loss: 0.0888 - acc: 0.9667 - val_loss: 0.3143 - val_acc: 0.9195
Epoch 7/10
4989/4989 [==============================] - 1s - loss: 0.0702 - acc: 0.9757 - val_loss: 0.3491 - val_acc: 0.9192
Epoch 8/10
4989/4989 [==============================] - 1s - loss: 0.0673 - a

In [189]:
loss_and_metrics = conv3.evaluate(test, y_test, batch_size=64);
print("the accuracy on test set is:",loss_and_metrics[1])

2816/3056 [==========================>...] - ETA: 0sthe accuracy on test set is: 0.914921466593


# Better ways to preprocess text

In [192]:
from keras.preprocessing.text import Tokenizer
EMBEDDING_DIM = 50
seq_len = 100
vocab_size = 5000

In [200]:
def extract_lines(file):
    count = 0
    lines = [];
    for line in file:
        lines.append(line)
    return lines

pos_tr = []
neg_tr = []

pos_te = []
neg_te = []


with open("./../fishing_data/pos.txt", "r", encoding="utf8") as pos_file:
    pos_tr = extract_lines(pos_file)
with open("./../fishing_data/neg.txt", "r", encoding="utf8") as pos_file:
    neg_tr = extract_lines(pos_file)
with open("./../fishing_data/test/pos.txt", "r", encoding="utf8") as pos_file:
    pos_te = extract_lines(pos_file)
with open("./../fishing_data/test/neg.txt", "r", encoding="utf8") as pos_file:
     neg_te = extract_lines(pos_file)

all_text = pos_tr + neg_tr + pos_te + neg_te

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(all_text)
sequences = tokenizer.texts_to_sequences(all_text)
word_index = tokenizer.word_index

In [220]:
sequences[0]

[15,
 294,
 82,
 47,
 11,
 40,
 5,
 100,
 151,
 109,
 13,
 14,
 2255,
 8,
 7,
 19,
 27,
 197,
 23,
 109,
 12,
 17,
 2364,
 113,
 81,
 3,
 117,
 101,
 12,
 671,
 131,
 61,
 5,
 11,
 327,
 47,
 119,
 6,
 146,
 5,
 15,
 294,
 48,
 4,
 5,
 15,
 294,
 3382,
 9,
 59,
 13,
 14,
 59,
 8,
 2255,
 252,
 1998,
 2769,
 3245,
 104,
 2710,
 2495,
 1376,
 5,
 15,
 294,
 701,
 545,
 47,
 20,
 575,
 47,
 20,
 594,
 61,
 20,
 594,
 4250,
 575,
 47,
 4250,
 545,
 20,
 20,
 4250,
 545,
 12,
 20,
 47,
 20,
 1596,
 327,
 141,
 35,
 15,
 923,
 569]

"\ufeffFishing with Rod: BC Stillwater Trout For more fishing videos, please visit: http://www.fishingwithrod.com    In this video blog, we visit Onion Lake at Ruddocks Ranch just north of Lytton, British Columbia. Onion Lake offers good stillwater fly fishing for rainbow trout.    Subscribe to go Fishing with Rod!    Be a Fishing with Rod fan on Facebook:  http://www.facebook.com/fishingwithrod    Camera: Nina Manique  Edit: Rodney Hsu  Music: Getty Images Inc.  Copyright: Fishing with Rod Production ['BC trout fishing', 'rainbow trout fishing', 'trout fly fishing', 'trout flyfishing', 'rainbow trout flyfishing', 'BC stillwater fishing', 'chironomid fishing', 'damsel flyfishing', 'BC lake fishing', 'stillwater trout fishing', 'trophy rainbow trout', 'Fishing with Rod', 'fishingwithrod.com']\n"

fishing
with
rod
bc
stillwater
trout
for
more
fishing
videos
please
visit
http
www
fishingwithrod
com
in
this
video
blog
we
visit
onion
lake
at
ruddocks
ranch
just
north
of
lytton
british
columbia
onion
lake
offers
good
stillwater
fly
fishing
for
rainbow
trout
subscribe
to
go
fishing
with
rod
!
be
a
fishing
with
rod
fan
on
facebook
http
www
facebook
com
fishingwithrod
camera
nina
manique
edit
rodney
hsu
music
getty
images
inc
copyright
fishing
with
rod
production
bc
trout
fishing
rainbow
trout
fishing
trout
fly
fishing
trout
flyfishing
rainbow
trout
flyfishing
bc
stillwater
fishing
chironomid
fishing
damsel
flyfishing
bc
lake
fishing
stillwater
trout
fishing
trophy
rainbow
trout
fishing
with
rod
fishingwithrod
com
